### Simple Intorduction
- SqlAlchemy is a python based ORM(Object Relationship Mapper) Tool that makes us intract with the database using python instead of actual SQL queries
- It consists of **TWO main parts** 
    - SqlAlchemy Core (Low Level)
    - SqlAlchemy ORM (High Level)

# SqlAlchemy Core

### Engine and Connection Creation

In [1]:
from sqlalchemy import create_engine, text

In [4]:
DATABASE_URL = "sqlite:///CRUDOperations.db"
engine = create_engine(DATABASE_URL, echo=True) # We can also create connection pools to efficiently make multiply connections
conn = engine.connect()

### Basic table creation and deletion query

**note**: Here in DDL(CREATE, ALTER, DROP) commands there is no need to commit, and rollback will have no effect

In [8]:
conn.execute(text("CREATE TABLE IF NOT EXISTS person (id int, name str, age int)"))

2025-09-19 23:42:52,396 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-19 23:42:52,398 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS person (id int, name str, age int)
2025-09-19 23:42:52,398 INFO sqlalchemy.engine.Engine [cached since 351.1s ago] ()


In [9]:
conn.execute(text("DROP TABLE person"))

2025-09-19 23:43:14,640 INFO sqlalchemy.engine.Engine DROP TABLE person
2025-09-19 23:43:14,642 INFO sqlalchemy.engine.Engine [generated in 0.00294s] ()


### Working on actual SQLAlchemy Core

In [13]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData

In [54]:
engine = create_engine(DATABASE_URL, echo=True)
meta = MetaData() # MetaData is basically a registry / container object that holds information about your database schema.
conn = engine.connect()

#### Table Creation

In [55]:
person = Table(
    "person",
    meta,
    Column("id", Integer, primary_key=True),
    Column("name", String, nullable=False),
    Column("age", Integer, nullable=False)
)

meta.create_all(engine)

2025-09-20 00:57:33,744 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 00:57:33,746 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("person")
2025-09-20 00:57:33,748 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-20 00:57:33,748 INFO sqlalchemy.engine.Engine COMMIT


#### Record Insertion

In [56]:
insert_statement = person.insert().values(name="Mike", age=30)
result = conn.execute(insert_statement)
conn.commit()

2025-09-20 00:57:34,319 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 00:57:34,319 INFO sqlalchemy.engine.Engine INSERT INTO person (name, age) VALUES (?, ?)
2025-09-20 00:57:34,322 INFO sqlalchemy.engine.Engine [generated in 0.00361s] ('Mike', 30)
2025-09-20 00:57:34,325 INFO sqlalchemy.engine.Engine COMMIT


In [57]:
from sqlalchemy import insert
insert_statement = insert(person).values(name="Tanvi", age=24)
result = conn.execute(insert_statement)
conn.commit()

2025-09-20 00:57:34,519 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 00:57:34,529 INFO sqlalchemy.engine.Engine INSERT INTO person (name, age) VALUES (?, ?)
2025-09-20 00:57:34,530 INFO sqlalchemy.engine.Engine [cached since 0.2111s ago] ('Tanvi', 24)
2025-09-20 00:57:34,534 INFO sqlalchemy.engine.Engine COMMIT


#### Record Selections

In [58]:
select_statement = person.select().where(person.c.age > 20)
result = conn.execute(select_statement)

for record in result:
    print(record)

2025-09-20 00:57:34,886 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 00:57:34,887 INFO sqlalchemy.engine.Engine SELECT person.id, person.name, person.age 
FROM person 
WHERE person.age > ?
2025-09-20 00:57:34,888 INFO sqlalchemy.engine.Engine [generated in 0.00253s] (20,)
(2, 'Tanvi', 24)
(3, 'Mike', 30)
(4, 'Randy', 42)
(5, 'Jhon', 28)
(6, 'Steve', 32)
(7, 'Ross', 24)
(8, 'Rachel', 22)
(9, 'Mike', 30)
(10, 'Tanvi', 24)


#### Updating Records

In [59]:
update_statement = person.update().where(person.c.name == "Mike").values(age=50)
conn.execute(update_statement)
conn.commit()

result = conn.execute(person.select())
for record in result:
    print(record)

2025-09-20 00:57:35,238 INFO sqlalchemy.engine.Engine UPDATE person SET age=? WHERE person.name = ?
2025-09-20 00:57:35,244 INFO sqlalchemy.engine.Engine [generated in 0.00155s] (50, 'Mike')
2025-09-20 00:57:35,250 INFO sqlalchemy.engine.Engine COMMIT
2025-09-20 00:57:35,257 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 00:57:35,258 INFO sqlalchemy.engine.Engine SELECT person.id, person.name, person.age 
FROM person
2025-09-20 00:57:35,258 INFO sqlalchemy.engine.Engine [generated in 0.00220s] ()
(2, 'Tanvi', 24)
(3, 'Mike', 50)
(4, 'Randy', 42)
(5, 'Jhon', 28)
(6, 'Steve', 32)
(7, 'Ross', 24)
(8, 'Rachel', 22)
(9, 'Mike', 50)
(10, 'Tanvi', 24)


#### Record Deletion

In [60]:
delete_statement = person.delete().where(person.c.name == "Mike")
conn.execute(delete_statement)
conn.commit()

result = conn.execute(person.select())
for record in result:
    print(record)

2025-09-20 00:57:35,835 INFO sqlalchemy.engine.Engine DELETE FROM person WHERE person.name = ?
2025-09-20 00:57:35,837 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ('Mike',)
2025-09-20 00:57:35,840 INFO sqlalchemy.engine.Engine COMMIT
2025-09-20 00:57:35,846 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 00:57:35,847 INFO sqlalchemy.engine.Engine SELECT person.id, person.name, person.age 
FROM person
2025-09-20 00:57:35,847 INFO sqlalchemy.engine.Engine [cached since 0.5916s ago] ()
(2, 'Tanvi', 24)
(4, 'Randy', 42)
(5, 'Jhon', 28)
(6, 'Steve', 32)
(7, 'Ross', 24)
(8, 'Rachel', 22)
(10, 'Tanvi', 24)


#### Create A Relation using ForeignKey

In [61]:
from sqlalchemy import ForeignKey, Float

things = Table(
    "things",
    meta,
    Column("id", Integer, primary_key=True),
    Column("description", String, nullable=False),
    Column("value", Float, nullable=False),
    Column("owner", Integer, ForeignKey("person.id"))
)

meta.create_all(bind=engine)

2025-09-20 00:57:36,188 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 00:57:36,191 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("person")
2025-09-20 00:57:36,191 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-20 00:57:36,192 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("things")
2025-09-20 00:57:36,194 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-20 00:57:36,195 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
insert_statement = person.insert().values([
    {"name":"Mike", "age":30},
    {"name":"Randy", "age":42},
    {"name":"Jhon", "age":28},
    {"name":"Steve", "age":32},
    {"name":"Ross", "age":24},
    {"name":"Rachel", "age":22},
])
conn.execute(insert_statement)
conn.commit()

In [63]:
insert_statement = things.insert().values([
    {"owner":2, "description":"Laptop", "value":800.50},
    {"owner":2, "description":"Mouse", "value":10.99},
    {"owner":2, "description":"KeyBoard", "value":20.99},
    {"owner":3, "description":"Speaker", "value":50.00},
    {"owner":4, "description":"Monitor", "value":40.30},
    {"owner":5, "description":"Mobile", "value":500.50},
])

conn.execute(insert_statement)
conn.commit()

2025-09-20 01:02:10,497 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 01:02:10,502 INFO sqlalchemy.engine.Engine INSERT INTO things (description, value, owner) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?)
2025-09-20 01:02:10,502 INFO sqlalchemy.engine.Engine [no key 0.00454s] ('Laptop', 800.5, 2, 'Mouse', 10.99, 2, 'KeyBoard', 20.99, 2, 'Speaker', 50.0, 3, 'Monitor', 40.3, 4, 'Mobile', 500.5, 5)
2025-09-20 01:02:10,509 INFO sqlalchemy.engine.Engine COMMIT


####  Join Statements

In [66]:
join_statement = person.join(things, person.c.id == things.c.owner)
select_statement = person.select().with_only_columns(person.c.name, things.c.description).select_from(join_statement)
result = conn.execute(select_statement)

for record in result:
    print(record)

2025-09-20 01:09:09,157 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 01:09:09,162 INFO sqlalchemy.engine.Engine SELECT person.name, things.description 
FROM person JOIN things ON person.id = things.owner
2025-09-20 01:09:09,164 INFO sqlalchemy.engine.Engine [generated in 0.00805s] ()
('Tanvi', 'Laptop')
('Tanvi', 'Mouse')
('Tanvi', 'KeyBoard')
('Randy', 'Monitor')
('Jhon', 'Mobile')


#### Group-BY Statements

In [70]:
from sqlalchemy import func
group_by_statement = things.select().with_only_columns(things.c.owner, func.sum(things.c.value)).group_by(things.c.owner)

result = conn.execute(group_by_statement)
for record in result:
    print(record)

2025-09-20 01:13:46,263 INFO sqlalchemy.engine.Engine SELECT things.owner, sum(things.value) AS sum_1 
FROM things GROUP BY things.owner
2025-09-20 01:13:46,267 INFO sqlalchemy.engine.Engine [generated in 0.00416s] ()
(2, 31.979999999999997)
(3, 50)
(4, 40.3)
(5, 500.5)


# SQLAlchemy ORM

In [45]:
from sqlalchemy import Column, String, Integer, ForeignKey, create_engine, func
from sqlalchemy.orm import sessionmaker, declarative_base, relationship

### Engine creation and connection establishment

In [2]:
DATABASE_URL = "sqlite:///CRUDOperations.db"
engine = create_engine(DATABASE_URL, echo=True)

### Table Creation

In [3]:
Base = declarative_base()

class User(Base):
    __tablename__ = "user"
    user_id: int = Column(Integer, primary_key=True)
    name: str = Column(String, nullable=False)
    email: str = Column(String, nullable=False)

    blogs = relationship("Blog", back_populates="author")

class Blog(Base):
    __tablename__ = "blog"
    blog_id: int = Column(Integer, primary_key=True)
    title: str = Column(String, nullable=False)
    description: str = Column(String, nullable=False)
    author_id: int = Column(Integer, ForeignKey("user.user_id"))

    author = relationship("User", back_populates="blogs")

In [4]:
Base.metadata.create_all(engine)

2025-09-19 17:39:02,412 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-19 17:39:02,414 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2025-09-19 17:39:02,415 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-19 17:39:02,417 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("blog")
2025-09-19 17:39:02,418 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-19 17:39:02,422 INFO sqlalchemy.engine.Engine COMMIT


### Session creation and accessing the data

In [5]:
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
new_user = User(name="Rashwanth", email="rashwanth@gmail.com")
session.add(new_user)
session.commit()

### Selecting data

In [22]:
kavin = session.query(User).filter(User.name=="Kavin").first()

2025-09-19 17:45:50,259 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?
2025-09-19 17:45:50,261 INFO sqlalchemy.engine.Engine [cached since 210.7s ago] ('Kavin', 1, 0)


In [24]:
all_record = session.query(User).all()

2025-09-19 17:47:01,719 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user
2025-09-19 17:47:01,722 INFO sqlalchemy.engine.Engine [cached since 475.5s ago] ()


In [26]:
new_blog = Blog(title="How to successfully lose in life!", description="Buy this one and dig a 6 foot grave and jump into it!", author_id=kavin.user_id)
session.add(new_blog)
session.commit()

2025-09-19 17:47:18,879 INFO sqlalchemy.engine.Engine INSERT INTO blog (title, description, author_id) VALUES (?, ?, ?)
2025-09-19 17:47:18,880 INFO sqlalchemy.engine.Engine [cached since 489.1s ago] ('How to successfully lose in life!', 'Buy this one and dig a 6 foot grave and jump into it!', 1)
2025-09-19 17:47:18,887 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
session.query(Blog).first().author.name

2025-09-19 17:42:28,260 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-19 17:42:28,262 INFO sqlalchemy.engine.Engine SELECT blog.blog_id AS blog_blog_id, blog.title AS blog_title, blog.description AS blog_description, blog.author_id AS blog_author_id 
FROM blog
 LIMIT ? OFFSET ?
2025-09-19 17:42:28,263 INFO sqlalchemy.engine.Engine [cached since 155.2s ago] (1, 0)
2025-09-19 17:42:28,266 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.user_id = ?
2025-09-19 17:42:28,268 INFO sqlalchemy.engine.Engine [cached since 145.7s ago] (1,)


'Kavin'

### Deleting and Updating Record

In [27]:
session.query(User).filter(User.name=="Rashwanth").delete()
session.commit()

2025-09-19 17:48:18,996 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-19 17:48:18,999 INFO sqlalchemy.engine.Engine DELETE FROM user WHERE user.name = ?
2025-09-19 17:48:19,000 INFO sqlalchemy.engine.Engine [generated in 0.00133s] ('Rashwanth',)
2025-09-19 17:48:19,007 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
session.query(User).filter(User.name=="Kavin").update({"email":"kavin@gmail.com"})
session.commit()

2025-09-19 17:49:39,288 INFO sqlalchemy.engine.Engine UPDATE user SET email=? WHERE user.name = ?
2025-09-19 17:49:39,289 INFO sqlalchemy.engine.Engine [cached since 16.54s ago] ('kavin@gmail.com', 'Kavin')
2025-09-19 17:49:39,292 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
### Joints

In [36]:
session.query(User.name, func.count(Blog.blog_id).label("Total No Blogs")).join(User.blogs).group_by(User.user_id).all()

2025-09-19 17:56:32,197 INFO sqlalchemy.engine.Engine SELECT user.name AS user_name, count(blog.blog_id) AS "Total No Blogs" 
FROM user JOIN blog ON user.user_id = blog.author_id GROUP BY user.user_id
2025-09-19 17:56:32,198 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ()


[('Kavin', 3)]